# 联邦学习贡献评估四种方法的代码实现

## 接口

输入：v：任务模型类型、m：评价方式如准确率、N：各参与方（的数据）

输出：各参与方的数据估值

## 读取与划分数据

In [ ]:
from loader.regression_data_loader import *
from loader.classification_data_loader import *
from loader.image_data_loader import *

In [ ]:
from measure.individual import Individual
from measure.leave_one_out import LeaveOneOut
from measure.shapley import Shapley
from measure.least_core import LeastCore

In [ ]:
from model.model import *

In [ ]:
loader = MNIST()

In [4]:
from dataset.Dataset import read_adult, uniform_split, ratio_split

X_train, y_train, X_test, y_test = read_adult(0.2)
X_train_3d, y_train_2d = uniform_split(3, X_train, y_train)

## 选择模型

In [7]:
from model.Model import return_model
model = return_model('SVC')

## 实现

### 个体法

In [9]:
from measure.INDIV import INDIV

measure = INDIV(X_train_3d, y_train_2d, X_test, y_test, model)
res = measure.score()

print(res)

[0.77 0.79 0.75]


### LOO(留一法)

https://github.com/AI-secure/Shapley-Study/blob/master/shapley/measures/LOO.py

In [10]:
from measure.LOO import LOO

measure = LOO(X_train_3d, y_train_2d, X_test, y_test, model)
res = measure.score()

print(res)

[0.03 0.06 0.  ]


### Shaply(夏普利值)

In [11]:
from measure.Shapley import Shapley

measure = Shapley(X_train_3d, y_train_2d, X_test, y_test, model)
res = measure.score()

print(res)
print(sum(res))

model.fit(X_train, y_train)
print(model.score(X_test, y_test))

[0.27333333 0.29833333 0.24833333]
0.8199999999999998
0.82


### 最小核

In [15]:
from measure.LeastCore import LeastCore

loader = Bank()
loader.read(nrows=700)
loader.encode()
loader.train_test_split(test_ratio=0.1)
# loader.ratio_split([0.5, 0.4, 0.1])
loader.ratio_split([0.05, 0.2, 0.25, 0.1, 0.2, 0.2])
loader.randomly_generate_data(40)
loader.reverse_y_train({0})
print(len(loader.y_train_parts))

model = return_model('LR')

measure = Shapley(loader, model)
res = measure.score()

# measure.plot()

print('res=', res)
print(sum(res))

model.fit(torch.cat(loader.X_train_parts), torch.cat(loader.y_train_parts))
print(model.score(loader.X_test, loader.y_test))

[0.27333333 0.29333333 0.25333333]
0.8200000000000001
0.82


1. 新增模型
3. 新增价值度量指标
4. 新增参与方数据分布方式

## 对方法的评估

性能评测的指标可以等先把上述流程跑通后再考虑，比如：
- 贡献评估准确率：一般是从全体下逐个按价值最高到最低移除参与方，最好的评估结果会下降最快
- 贡献评估耗时：评估需要耗时多久
- 能否抵御恶意参与方：比如输入数据中有参与方为随机数据、复制数据后，是否会受影响